In [2]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import importlib
import sqlite3

# keeping company information in additional file
import data_file
importlib.reload(data_file)

# Get current year and month
now = datetime.now()

# Subtract one month from current date
last_month = now - relativedelta(months=1)  #

# Format last month as string in YYYYMM format
year_month = last_month.strftime("%Y%m")

print(year_month)

# Construct file path
file_path = f"data_files/{year_month}.xlsx"

# Read xlsx file into pandas DataFrame
df = pd.read_excel(file_path)

202305


In [3]:
my_company_name = data_file.my_company_name
exclusion_company_one = data_file.exclusion_company_one
exclusion_company_two = data_file.exclusion_company_two

last_column_name = 'order_intake_amount_eur'
df = df.rename(columns={df.columns[-1]: last_column_name})

# necessary modifications of columns

df['bu'] = df['bu'].astype(str)
df['customer_name'] = df['sold_to_customer_n'].str.upper()

# Filter out rows with unwanted value in customer_name
df = df[~df['customer_name'].str.contains(my_company_name) & ~df['customer_name'].str.contains(exclusion_company_one) & ~df['customer_name'].str.contains(exclusion_company_two)]

# Drop rows where 'total' value is 0
df = df.drop(df[df[last_column_name] == 0].index)

# Get 'sales_order_so' where the sum of 'order_intake_amount_eur' is 0
result = df.groupby('sales_order_so')[last_column_name].transform('sum') == 0
sales_order_with_sum_zero = df[result]['sales_order_so'].unique().tolist()
df = df[~df['sales_order_so'].isin(sales_order_with_sum_zero)]

In [4]:
# visual check - sums should be the same or very close ~ 1
df.groupby(['source'])['order_intake_amount_eur'].sum()[0] / df.groupby(['source'])['order_intake_amount_eur'].sum()[1]

1.0025514695682574

In [5]:
df.groupby(['source'])['order_intake_amount_eur'].sum()[0]

28020302

In [6]:
df.groupby(['source'])['order_intake_amount_eur'].sum()[1]

27948991

In [7]:
#combining bu products groups for PRJ and BU sources
prj_bu = data_file.prj_bu
prd_bu = data_file.prd_bu

# delete doubled rows from difficrent sources PRJ and BU
list_prj_so = df[df['bu'].isin(prj_bu)]['sales_order_so'].to_list()
df_wo_prj_so = df[(~df['sales_order_so'].isin(list_prj_so)) & (df['source'] == 'BU')]
df_with_prj_so = df[df['bu'].isin(prj_bu)]

# keeping only unique sales orders
df_final = pd.concat([df_wo_prj_so, df_with_prj_so])

In [8]:
# visual check - sums should be the same or very close ~ 1
df_final['order_intake_amount_eur'].sum() / df.groupby(['source'])['order_intake_amount_eur'].sum()[0]

1.0000000356884091

In [9]:
# Create an ExcelWriter object
writer = pd.ExcelWriter(f'data_files/outcome/month_results_table_{year_month}.xlsx')
# Save each DataFrame to a separate sheet in the same file
df_final.to_excel(writer, sheet_name=f'{year_month}', index=False)
# Save the file
writer.close()

In [10]:
len(df_final)

2878

In [27]:
import numpy as np
import sqlite3
conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"  # Replace 'tablename' with your table name
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

print(len(df_customers))

14802


In [12]:
# Extract the difference by subtracting the common part

df_customers['legal_form'] = ''
df_customers['sold_to_customer_n'] = df_customers['sold_to_customer_n'].str.upper()
for i, row in df_customers.iterrows():
    common_part = row['customer_name']
    if common_part in row['sold_to_customer_n']:
        df_customers.at[i, 'legal_form'] = row['sold_to_customer_n'].replace(common_part, '').strip()

legal_forms = df_customers['legal_form'].tolist()
legal_forms = list(set(legal_forms))

In [13]:
list_of_existing_customer = df_customers['sold_to_customer'].tolist()
new_customers = df_final[~df_final['sold_to_customer'].isin(list_of_existing_customer)]

In [14]:
# reading table of definition for customer types
def_for_customers = pd.read_excel('data_files/def_for_customers.xlsx')

In [15]:
new_customers = new_customers.loc[:, ['sold_to_customer', 'company_code_n', 'sold_to_customer_n', 'customer_name','customer_group_code']]
new_customers.drop_duplicates(inplace=True)
new_customers['customer_group_code'] = new_customers['customer_group_code'].fillna(0)
new_customers = new_customers.merge(def_for_customers, how='left')

new_customers['customer_name_prev'] = new_customers['customer_name']
new_customers['legal_form'] = ''
for i, row in new_customers.iterrows():
    common_part = row['customer_name']
    coinc = []
    for lf in legal_forms:
        if lf in common_part:
            coinc.append(lf)
    
    longest_element = max(coinc, key=len)
    new_customers.at[i, 'customer_name'] = row['customer_name'].replace(longest_element, '').strip()
    new_customers.at[i, 'legal_form'] = longest_element

# Create an ExcelWriter object
writer = pd.ExcelWriter(f'data_files/outcome/new_customers_{year_month}.xlsx')
# Save each DataFrame to a separate sheet in the same file
new_customers.to_excel(writer, sheet_name='new_customers', index=False)
# Save the file
writer.close()

In [ ]:

# NOW it is necessary to open the file and check legal forms determination, then save the file and read it again


In [16]:
# after visual check of legal forms separation
checked_new_customers = pd.read_excel(f'data_files/outcome/new_customers_{year_month}.xlsx')

new_customer_list = checked_new_customers['customer_name'].tolist()

conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers" 
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

df_customers_to_check = df_customers[df_customers['customer_name'].isin(new_customer_list)]

existing_list = list(set(df_customers_to_check['customer_name'].tolist()))

existing_in_new = checked_new_customers[checked_new_customers['customer_name'].isin(existing_list)]
print(len(existing_in_new))

checked_new_customers = checked_new_customers[~checked_new_customers['customer_name'].isin(existing_list)]
print(len(checked_new_customers))

existing_in_new = existing_in_new.loc[:, ['sold_to_customer', 'company_code_n', 'sold_to_customer_n',
       'customer_name', 'customer_group_code'] ]


df_customers_for_use = df_customers.loc[:, ['customer_name', 'indirect_direct', 'channel',
       'type', 'tier']]

df_customers_for_use.drop_duplicates(inplace=True)

existing_in_new = existing_in_new.merge(df_customers_for_use, how='left')

print(len(existing_in_new))

checked_new_customers = pd.concat([checked_new_customers, existing_in_new])
print(len(checked_new_customers))

2
13
4
17


In [17]:
duplicated_rows = checked_new_customers[checked_new_customers['customer_name'].duplicated(keep=False)]
if len(duplicated_rows) > 0:
    writer = pd.ExcelWriter(f'data_files/outcome/check_doubled.xlsx')
    checked_new_customers.to_excel(writer, sheet_name='doubled', index=False)
    writer.close()

In [24]:
checked_new_customers = checked_new_customers.loc[:, 'sold_to_customer':]

In [28]:
writer = pd.ExcelWriter(f'data_files/outcome/df_customers_plus_this_month.xlsx')
df_customers.to_excel(writer, sheet_name='doubled', index=False)
writer.close()

In [21]:
indexes_to_drop = [14, 15]

for i in indexes_to_drop:
    checked_new_customers = checked_new_customers.drop(i) # check shift...

In [26]:
checked_new_customers = checked_new_customers.rename(columns={'customer_group_code':'code'})
checked_new_customers['Comments'] = year_month

df = checked_new_customers.copy()

# Connect to the SQLite database
conn = sqlite3.connect('data_files/customer_data.db')
table_name = 'customers'

existing_table = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 0", conn)

# Find the columns in the DataFrame that are not in the existing table
new_columns = [column for column in df.columns if column not in existing_table.columns]

# Add the missing columns to the existing table (alter table query)
for column in new_columns:
    dtype = str(df[column].dtype)
    alter_query = f"ALTER TABLE {table_name} ADD COLUMN {column} {dtype}"
    conn.execute(alter_query)
    
    
# Insert the DataFrame data into the existing table
df.to_sql(table_name, conn, if_exists='append', index=False)

# Close the connection to the database
conn.close()

In [29]:
# update orders of last period according to the initial order table
df_final = df_final.drop('customer_name', axis=1)
df_final = df_final.rename(columns={'bu':'bu2'})
df_final['Date'] = pd.to_datetime(df_final['year_month'], format='%Y%m')
df_final['FY'] = df_final['Date'] - pd.DateOffset(months=3)

In [30]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"
df_orders = pd.read_sql_query(query, conn)
# Close the connection to the database
conn.close()

print(len(df_orders))

204559


In [31]:
# clean data base from entries for this period

print(year_month)

def delete_rows(db_file, table_name, column_name, value):
    # Connect to the database
    conn = sqlite3.connect(db_file)
    cursor = conn.cursor()

    # Execute the DELETE statement to remove rows
    delete_query = f"DELETE FROM {table_name} WHERE {column_name} = ?"
    cursor.execute(delete_query, (value,))

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

# Provide the necessary details
database_file = "data_files/order_data.db"
table_name = "orders"
column_name = "year_month"
value = year_month

# Call the function to delete rows
delete_rows(database_file, table_name, column_name, value)

202305


In [32]:
# updating db with orders

conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"  # Replace 'tablename' with your table name

#df_orders = pd.read_sql_query(query, conn)

#copy df with new results
df = df_final.copy()

table_name = 'orders'

existing_table = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 0", conn)

# Find the columns in the DataFrame that are not in the existing table
new_columns = [column for column in df.columns if column not in existing_table.columns]

# Add the missing columns to the existing table (alter table query)
for column in new_columns:
    dtype = str(df[column].dtype)
    alter_query = f"ALTER TABLE {table_name} ADD COLUMN {column} {dtype}"
    conn.execute(alter_query)
    
    
# Insert the DataFrame data into the existing table
df.to_sql(table_name, conn, if_exists='append', index=False)

# Close the connection to the database
conn.close()

In [33]:
conn = sqlite3.connect('data_files/order_data.db')
query = "SELECT * FROM orders"
df_orders = pd.read_sql_query(query, conn)
# Close the connection to the database
conn.close()

print(len(df_orders))

205248


In [34]:
writer = pd.ExcelWriter(f'data_files/outcome/df_orders_check.xlsx')
df_orders.to_excel(writer, sheet_name='orders', index=False)
writer.close()

In [28]:
conn2 = sqlite3.connect('data_files/customer_data.db')
query = "SELECT * FROM customers"  # Replace 'tablename' with your table name
df_customers = pd.read_sql_query(query, conn2)
conn2.close()

writer = pd.ExcelWriter(f'data_files/outcome/df_customers_check.xlsx')
df_customers.to_excel(writer, sheet_name='doubled', index=False)
writer.close()